In [1]:
# Importar las bibliotecas necesarias
%load_ext autoreload
%autoreload 2
import sys
# Añadimos la carpeta que contiene nuestro .py al path de Python
sys.path.append("../src/")
import support_tag as st
import support_mongo as sm
import pandas as pd

In [3]:
# Importar datos bbdd
bd=sm.conectar_a_mongo("PullnBearData")
nombre_coleccion1="modelos_pull_hombre"
nombre_coleccion2="productos_pull_hombre"
df_modelos = sm.importar_a_dataframe(bd, nombre_coleccion1)
df_productos = sm.importar_a_dataframe(bd, nombre_coleccion2)

In [4]:
# Filtrar productos que son "Pantalón"
df_productos_inferiores = df_productos[df_productos['Categoria'] == 'Pantalón']

# Filtrar productos que NO son "Pantalón"
df__productos_superiores = df_productos[df_productos['Categoria'] != 'Pantalón']

In [5]:
df_modelos1=df_modelos.sample(2)
df_productos_inferiores1=df_productos_inferiores.sample(2)
df__productos_superiores1=df__productos_superiores.sample(2)

### Tagear Modelos

In [ ]:
import requests
from io import BytesIO
import base64

def download_image(image_url):
    try:
        response = requests.get(image_url, timeout=5)  # Establece un tiempo de espera
        response.raise_for_status()  # Lanza un error si la descarga falla
        
        content_type = response.headers.get("Content-Type", "")
        if "image" not in content_type:
            print(f"La URL no es una imagen válida: {image_url}")
            return None
        
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar la imagen {image_url}: {e}")
        return None


image_url="https://static.pullandbear.net/assets/public/104d/71ac/19744008911a/0730d644dece/07678525802-M/07678525802-M.jpg?ts=1730136321453&w=438&f=auto"
download_image(image_url)


In [7]:
df_modelos_tageados = st.tag_models(df_modelos1)

In [9]:
df_modelos_tageados.to_csv('../results/Modelos_pruebas.csv', index=False)

In [ ]:
df_productos_superior_tageados=st.tag_products_superior(df__productos_superiores1)

In [ ]:
df_productos_inferiores_tageados=st.tag_products_inferior(df_productos_inferiores1)

In [16]:
df_productos_tageados=pd.concat([df_productos_superior_tageados,df_productos_inferiores_tageados], ignore_index=True)

In [18]:
df_productos_tageados.to_csv('../results/productos_pruebas.csv', index=False)

In [ ]:
# Guardamos los datos en la bbdd de Mongo
bd=sm.conectar_a_mongo("PullnBearData")
nombre_coleccion1="modelos_pull_hombre_tageados"
nombre_coleccion2="productos_pull_hombre_tageados"
sm.eliminar_coleccion(bd, nombre_coleccion1)
sm.subir_dataframe_a_mongo(bd, df_modelos_tageados, nombre_coleccion1)
sm.eliminar_coleccion(bd, nombre_coleccion2)
sm.subir_dataframe_a_mongo(bd, df_productos_tageados, nombre_coleccion2)